### Loading Datasets

In [1]:
import re
import numpy as np 
import pandas as pd
pd.set_option('display.max_rows', 700)
from sklearn.utils import shuffle

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
train = shuffle(pd.read_json('is_train.json'), random_state=0)
val = shuffle(pd.read_json('is_val.json'), random_state=0)
test = pd.read_json('is_test.json')
oos_train = pd.read_json('oos_train.json')
oos_val = pd.read_json('oos_val.json')
oos_test = pd.read_json('oos_test.json')

files = [
         (train,'train'),
         (val,'val'),
         (test,'test'),
         (oos_train,'oos_train'),
         (oos_val,'oos_val'),
         (oos_test,'oos_test')
         ]
         
for file,name in files:
    file.columns = ['text','intent']
    print(f'{name} shape:{file.shape}, {name} has {train.isna().sum().sum()} null values')
in_train = train.copy()

train shape:(15000, 2), train has 0 null values
val shape:(3000, 2), val has 0 null values
test shape:(4500, 2), test has 0 null values
oos_train shape:(100, 2), oos_train has 0 null values
oos_val shape:(100, 2), oos_val has 0 null values
oos_test shape:(1000, 2), oos_test has 0 null values


In [48]:
in_train = pd.concat([in_train, oos_train], axis=0).reset_index(drop=True)
val = pd.concat([val, oos_val], axis=0).reset_index(drop=True)
test = pd.concat([test, oos_test], axis=0).reset_index(drop=True)

In [49]:
in_train = shuffle(in_train , random_state=0).reset_index(drop=True)
val = shuffle(val, random_state=0).reset_index(drop=True)
test = shuffle(test, random_state=0).reset_index(drop=True)

In [50]:
in_train.shape

(15100, 2)

In [51]:
in_train

,text,intent
0,is it possible for you to use the whisper voice,whisper_mode
1,search for a macaroni and cheese recipe,recipe
2,correct,yes
3,"my card got scratched badly, can you tell my b...",damaged_card
4,i'd like to know what my credit rating is,credit_score
...,...,...
15095,please play come as you are by nirvana,play_music
15096,"book a hotel in pa, harrisburg for the 10th to...",book_hotel
15097,what kind of mpg does this car get in the city,mpg
15098,roll dice,roll_dice


In [52]:
in_train.intent.value_counts()

w2                           100
schedule_maintenance         100
measurement_conversion       100
vaccines                     100
carry_on                     100
pto_request                  100
transactions                 100
order                        100
shopping_list                100
distance                     100
transfer                     100
interest_rate                100
change_user_name             100
reminder                     100
bill_due                     100
what_are_your_hobbies        100
calculator                   100
reminder_update              100
current_location             100
oil_change_when              100
order_checks                 100
international_fees           100
no                           100
change_accent                100
rollover_401k                100
share_location               100
income                       100
cancel_reservation           100
who_do_you_work_for          100
book_hotel                   100
meal_sugge

In [53]:
in_train['intent'].nunique(), val['intent'].nunique(), test['intent'].nunique()

(151, 151, 151)

In [54]:
# oss_plus_train = pd.concat([in_train, oos_train],axis=0).reset_index(drop=True)
# oss_plus_val = pd.concat([val,oos_val],axis=0).reset_index(drop=True)
# oss_plus_test = pd.concat([test,oos_test],axis=0).reset_index(drop=True)

In [55]:
# oss_plus_train['intent'].nunique(), oss_plus_val['intent'].nunique(), oss_plus_test['intent'].nunique()

In [56]:
# def binarize(df):
#     df.intent = np.where(df.intent!='oos',0,1)
#     return df

In [57]:
# oos_plus_train = binarize(pd.concat([in_train,oos_train],axis=0).reset_index(drop=True))
# oos_plus_val = binarize(pd.concat([val,oos_val],axis=0).reset_index(drop=True))
# oos_plus_test = binarize(pd.concat([test,oos_test],axis=0).reset_index(drop=True))

In [58]:
# oos_count = oos_plus_train.intent.value_counts()
# oos_count

In [59]:
# oss_plus_train.head()

In [60]:
from sklearn.preprocessing import LabelBinarizer
labelBinary = LabelBinarizer()
 
labels = labelBinary.fit_transform(in_train['intent'])
text = in_train['text']
# import numpy as np
# val = [
#        [0, 0, 1],
#        [1, 0, 0]
# ]
 
# lb.inverse_transform(np.asarray(val))

In [61]:
labelBinary.classes_

array(['accept_reservations', 'account_blocked', 'alarm',
       'application_status', 'apr', 'are_you_a_bot', 'balance',
       'bill_balance', 'bill_due', 'book_flight', 'book_hotel',
       'calculator', 'calendar', 'calendar_update', 'calories', 'cancel',
       'cancel_reservation', 'car_rental', 'card_declined', 'carry_on',
       'change_accent', 'change_ai_name', 'change_language',
       'change_speed', 'change_user_name', 'change_volume',
       'confirm_reservation', 'cook_time', 'credit_limit',
       'credit_limit_change', 'credit_score', 'current_location',
       'damaged_card', 'date', 'definition', 'direct_deposit',
       'directions', 'distance', 'do_you_have_pets', 'exchange_rate',
       'expiration_date', 'find_phone', 'flight_status', 'flip_coin',
       'food_last', 'freeze_account', 'fun_fact', 'gas', 'gas_type',
       'goodbye', 'greeting', 'how_busy', 'how_old_are_you',
       'improve_credit_score', 'income', 'ingredient_substitution',
       'ingredients_l

In [62]:
import pickle

labelpath = 'label-151.pkl'
with open(labelpath, 'wb') as handle:
    pickle.dump(labelBinary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [63]:
print(labels.shape, text.shape)

(15100, 151) (15100,)


In [64]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

#### Transform validation dataset

In [65]:
test.head()

,text,intent
0,i would like to do some things in phoenix,travel_suggestion
1,what are the steps to rollover my 401k,rollover_401k
2,what films are in the running for best song at...,oos
3,will you tell me how the ai is doing,greeting
4,how would i say what is your name if i were fr...,translate


In [66]:
val_labels = labelBinary.transform(test['intent'])
val_text = test['text']

In [67]:
val_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### **Download & Load Word Vectors for Embedding (Glove)**

* *Download and extract word vectors Glove*

In [ ]:
# %%time

import zipfile
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
zip_file = zipfile.ZipFile('glove.840B.300d.zip')
zip_file.extractall()

--2022-01-05 07:49:27--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2022-01-05 07:49:27--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2022-01-05 07:49:27--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

* *Load Word vectors in memory*

In [28]:
%%time

from tqdm.notebook import tqdm
import numpy as np 
 
def get_coefs(word, *arr):
    try:
        return word, np.asarray(arr, dtype='float32')
    except:
        return None, None

embedding_path = '/content/drive/MyDrive/data_science/embeddings/glove.840B.300d.txt'
embeddings_index = dict(get_coefs(*o.strip().split()) for o in tqdm(open(
    embedding_path)))
 
embed_size=300
 
for k in tqdm(list(embeddings_index.keys())):
    v = embeddings_index[k]
    try:
        if v.shape != (embed_size, ):
            embeddings_index.pop(k)
    except:
        pass
 
if None in embeddings_index:
  embeddings_index.pop(None)
  
values = list(embeddings_index.values())
all_embs = np.stack(values)
 
emb_mean, emb_std = all_embs.mean(), all_embs.std()

0it [00:00, ?it/s]

  0%|          | 0/2195885 [00:00<?, ?it/s]

CPU times: user 3min 13s, sys: 15.1 s, total: 3min 28s
Wall time: 3min 41s


### **Tokenizing the Dataset using keras**



* *Read the train and test file from directory and split the train dataset in train test split*

In [29]:
import tensorflow

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
 
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
 
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from tensorflow.keras.layers import Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import SpatialDropout1D, concatenate
from tensorflow.keras.layers import GRU,LSTM,Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import TimeDistributed
 
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#### **Save Keras Tokenizer**

In [68]:
text.map(len).max()

136

In [69]:
%%time
MAX_NB_WORDS = 10000
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)

CPU times: user 229 ms, sys: 1.99 ms, total: 231 ms
Wall time: 233 ms


In [70]:
%%time
train_sequences = tokenizer.texts_to_sequences(text)

CPU times: user 178 ms, sys: 743 µs, total: 179 ms
Wall time: 183 ms


In [71]:
train_sequences[:3]

[[10, 24, 257, 8, 5, 2, 75, 4, 211, 188],
 [664, 8, 7, 665, 27, 584, 283],
 [536]]

In [72]:
print(text[0], train_sequences[0])

is it possible for you to use the whisper voice [10, 24, 257, 8, 5, 2, 75, 4, 211, 188]


In [73]:
def FindMaxLength(lst): 
    maxList = max(lst, key = lambda i: len(i)) 
    maxLength = len(maxList) 
    return maxLength

MAX_LENGTH = FindMaxLength(train_sequences)

In [74]:
print(MAX_LENGTH)

28


* Now the Sentences are mapped to lists of integers. However, we still cannot stack them together in a matrix since they have different lengths.
Hopefully Keras allows to **pad** sequences with **0s** to a maximum length. We'll set this length to 425.

In [75]:
%%time
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)

CPU times: user 90.3 ms, sys: 6.36 ms, total: 96.7 ms
Wall time: 89.1 ms


In [76]:
%%time
padded_train_sequences

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


array([[  0,   0,   0, ...,   4, 211, 188],
       [  0,   0,   0, ...,  27, 584, 283],
       [  0,   0,   0, ...,   0,   0, 536],
       ...,
       [  0,   0,   0, ...,  15,   4, 303],
       [  0,   0,   0, ...,   0, 165, 221],
       [  0,   0,   0, ...,   6,  22, 210]], dtype=int32)

In [77]:
%%time
padded_train_sequences.shape

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 10.5 µs


(15100, 28)

In [78]:
import pickle
 
with open('token-28.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### **Use Saved keras tokenizer**

In [79]:
import pickle
cat_token = 'token-28.pkl'
with open(cat_token, 'rb') as handle:
    load_token = pickle.load(handle)

In [80]:
text[2]

'correct'

In [81]:
# MAX_LENGTH = 28
train_sequences = load_token.texts_to_sequences(text)
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)

# padding val
val_sequences = load_token.texts_to_sequences(val_text)
padded_val_sequences = pad_sequences(val_sequences, maxlen=MAX_LENGTH)

In [82]:
padded_train_sequences.shape

(15100, 28)

In [83]:
padded_val_sequences.shape

(5500, 28)

### **Training Model using Glove Embedding while using RNN & CNN**

In [84]:
%%time
# MAX_NB_WORDS = 2600
word_index = load_token.word_index
nb_words = MAX_NB_WORDS
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
 
oov = 0
for word, i in tqdm(word_index.items()):
    if i >= MAX_NB_WORDS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        oov += 1
 
print(oov)

  0%|          | 0/5350 [00:00<?, ?it/s]

340
CPU times: user 186 ms, sys: 2.01 ms, total: 188 ms
Wall time: 194 ms


* Loading the Layers and Embedding

In [85]:
len(labels[0])

151

In [86]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [87]:
len(val_labels[0])

151

In [88]:
LABEL_VALUES = labels
LABEL_LEN = len(labels[0])
# MAX_LENGTH = 1155
# MAX_NB_WORDS = 1170

In [89]:
LABEL_LEN

151

In [90]:
%%time
from tensorflow.keras.regularizers import l2

def get_rnn_cnn_model():
    embedding_dim = 300
    inp = Input(shape=(MAX_LENGTH, ))
    x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    # x = Bidirectional(LSTM(100, return_sequences=True))(x)
    # x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_regularizer=l2(0.0005))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(LABEL_LEN, activation="softmax")(conc)
    model = Model(inputs=inp, outputs=outp)
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

rnn_cnn_model = get_rnn_cnn_model()

CPU times: user 1.08 s, sys: 3.18 s, total: 4.26 s
Wall time: 6.35 s


* **Training and Saving the Model**

* *change mode to min if we are going to monitor the loss*
* *If we don't want to use validation data for whatever be the reason and implement check pointing, we have to change the ModelCheckpoint to work based on monitor='acc'*
* We have to fit validation data like this validation_data=(X_valid, y_valid),

In [91]:
filepath = 'oos_model.h5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
 
# batch size factor of 4,8,16,32
BATCH_SIZE = 256
epochs = 100

In [92]:
history = rnn_cnn_model.fit(padded_train_sequences, 
                    LABEL_VALUES, 
                    # validation_split = 0.1, #0.1
                    validation_data=(padded_val_sequences, val_labels), 
                    batch_size=BATCH_SIZE, 
                    callbacks=[checkpoint,
                               tensorflow.keras.callbacks.EarlyStopping(
                                   monitor='val_accuracy', patience=3)], 
                    epochs=epochs, 
                    verbose=1,
                    shuffle=True)

Epoch 1/100
59/59 [==============================] - ETA: 0s - loss: 4.7836 - accuracy: 0.0756
Epoch 00001: val_accuracy improved from -inf to 0.21382, saving model to oos_model.h5
59/59 [==============================] - 20s 71ms/step - loss: 4.7836 - accuracy: 0.0756 - val_loss: 4.1632 - val_accuracy: 0.2138
Epoch 2/100
58/59 [============================>.] - ETA: 0s - loss: 2.6180 - accuracy: 0.4657
Epoch 00002: val_accuracy improved from 0.21382 to 0.53527, saving model to oos_model.h5
59/59 [==============================] - 3s 52ms/step - loss: 2.5989 - accuracy: 0.4694 - val_loss: 2.0302 - val_accuracy: 0.5353
Epoch 3/100
58/59 [============================>.] - ETA: 0s - loss: 0.9393 - accuracy: 0.7952
Epoch 00003: val_accuracy improved from 0.53527 to 0.69127, saving model to oos_model.h5
59/59 [==============================] - 3s 52ms/step - loss: 0.9353 - accuracy: 0.7960 - val_loss: 1.3910 - val_accuracy: 0.6913
Epoch 4/100
58/59 [============================>.] - ETA: 0s

### **Loading the Tokenizer and Model**

In [93]:
from keras.models import load_model
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import pickle

In [94]:
modelPath = 'oos_model.h5'
tokenPath = 'token-28.pkl'
labelPath = '/content/label-151.pkl'

In [95]:
loadModel= load_model(modelPath)

with open(tokenPath, 'rb') as handle:
    loadToken = pickle.load(handle)

with open(labelPath, 'rb') as handle:
    loadLabel = pickle.load(handle)


In [96]:
LABEL = loadLabel.classes_

In [97]:
LABEL

array(['accept_reservations', 'account_blocked', 'alarm',
       'application_status', 'apr', 'are_you_a_bot', 'balance',
       'bill_balance', 'bill_due', 'book_flight', 'book_hotel',
       'calculator', 'calendar', 'calendar_update', 'calories', 'cancel',
       'cancel_reservation', 'car_rental', 'card_declined', 'carry_on',
       'change_accent', 'change_ai_name', 'change_language',
       'change_speed', 'change_user_name', 'change_volume',
       'confirm_reservation', 'cook_time', 'credit_limit',
       'credit_limit_change', 'credit_score', 'current_location',
       'damaged_card', 'date', 'definition', 'direct_deposit',
       'directions', 'distance', 'do_you_have_pets', 'exchange_rate',
       'expiration_date', 'find_phone', 'flight_status', 'flip_coin',
       'food_last', 'freeze_account', 'fun_fact', 'gas', 'gas_type',
       'goodbye', 'greeting', 'how_busy', 'how_old_are_you',
       'improve_credit_score', 'income', 'ingredient_substitution',
       'ingredients_l

In [98]:
len(LABEL)

151

### **Testing**

In [ ]:
# oss_plus_val.head()

In [ ]:
# test = pd.read_csv('/content/test.tsv', sep='\t')

In [99]:
val

,text,intent
0,how much do i need to pay in tax,taxes
1,forget reservation at longhorn for 4,cancel_reservation
2,is rice bad after 3 days in fridge,food_last
3,i want to change your name to sarah,change_ai_name
4,what do i get paid,income
...,...,...
3095,call aj now,make_call
3096,i need my earned income form for paying taxes,w2
3097,can i travel to france as far as safety goes,travel_alert
3098,read my reminder list to me please,reminder


In [100]:
val.shape

(3100, 2)

In [101]:
test_label = val['intent'].values
sentence = val['text'].values

In [102]:
# tokenization
maxLength = 28
test_sequences = loadToken.texts_to_sequences(sentence)
padded_test_sequences = pad_sequences(test_sequences, maxlen=maxLength)

In [103]:
# predection
pred_ls = loadModel.predict(padded_test_sequences)
# print(type(pred_ls))
category = loadLabel.inverse_transform(pred_ls)
# score = ('{:.2f}'.format(round(np.max(pred_ls), 2)*100))

In [104]:
pred_ls

array([[5.4298268e-09, 7.8544504e-09, 3.1149812e-11, ..., 8.4993673e-08,
        5.7188598e-10, 1.7556124e-13],
       [7.3705651e-02, 3.5163275e-10, 2.1715647e-05, ..., 8.3547863e-10,
        6.2043787e-11, 5.1300483e-12],
       [8.1172580e-08, 8.7592220e-08, 3.0545046e-08, ..., 1.5034976e-10,
        1.6796842e-13, 2.2870384e-11],
       ...,
       [3.2190763e-04, 1.4926925e-07, 9.8239681e-08, ..., 1.5214505e-08,
        2.4250883e-06, 8.2970783e-12],
       [4.1752391e-12, 1.1860695e-06, 1.3775455e-06, ..., 1.7249565e-11,
        4.5649383e-12, 2.8880557e-13],
       [2.5662854e-07, 1.2240994e-08, 7.0751063e-05, ..., 1.1901884e-09,
        1.0817330e-10, 2.6795371e-10]], dtype=float32)

In [105]:
# score = []
# for i in pred_ls:
#   score.append(np.max(pred_ls)*100)

In [106]:
category

array(['taxes', 'cancel_reservation', 'food_last', ..., 'vaccines',
       'reminder', 'reminder_update'], dtype='<U25')

In [107]:
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score

# DeepAlingned : NMI: 93.86, ARI: 79.75, ACC: 86.49 
print('ACC : ', round(accuracy_score(test_label, category)*100, 2))
print('ARI : ', round(adjusted_rand_score(test_label, category)*100, 2))
print('NMI : ', round(normalized_mutual_info_score(test_label, category)*100, 2))

ACC :  87.97
ARI :  73.19
NMI :  91.39


In [ ]:
# result = []
# for i in pred_ls:
#     if i > 0.5:
#         result.append(1)
#     else:
#         result.append(0)

### **Accuracy, ROC AUC, Confusion Matrix and Classification Report**

In [108]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import matthews_corrcoef
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
sns.set(rc={'figure.facecolor':'white'})

# input values
TEST_LABEL = test_label
PREDICTED_LABEL = category
labels = LABEL

# accuracy = accuracy_score(TEST_LABEL, PREDICTED_LABEL)
# conf_mat = confusion_matrix(TEST_LABEL, PREDICTED_LABEL)
# fig, ax = plt.subplots(figsize=(30,27))
# sns.heatmap(conf_mat, annot=True, fmt='d',
#             xticklabels=labels, yticklabels=labels)
# plt.ylabel('Actual Label')
# # plt.xlabel('Predicted Label\n\nAccuracy={:0.4f}; ROC AUC={:0.4f}'.format(accuracy, roc_score))
# plt.xlabel('Predicted Label\n\nAccuracy={:0.4f}'.format(accuracy))
# # plt.savefig('/content/drive/My Drive/Lafarge/Model/Hardware/HardwareModel-Confusion-v1.5.jpg')
# # plt.savefig('HardwareModel-Confusion-v1.5.jpg')
# plt.xticks(rotation=90)
# plt.show()

In [109]:
from sklearn.metrics import classification_report
print(classification_report(test_label, category, target_names=LABEL))

                           precision    recall  f1-score   support

      accept_reservations       0.84      0.80      0.82        20
          account_blocked       0.82      0.70      0.76        20
                    alarm       0.95      1.00      0.98        20
       application_status       0.90      0.90      0.90        20
                      apr       1.00      0.95      0.97        20
            are_you_a_bot       0.90      0.95      0.93        20
                  balance       0.64      0.90      0.75        20
             bill_balance       0.94      0.80      0.86        20
                 bill_due       0.80      1.00      0.89        20
              book_flight       1.00      1.00      1.00        20
               book_hotel       0.87      1.00      0.93        20
               calculator       0.73      0.95      0.83        20
                 calendar       0.94      0.75      0.83        20
          calendar_update       0.90      0.95      0.93     